In [16]:
%pip install azure-storage-blob
%pip install azure-search-documents
%pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta
import requests
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv
import os

In [18]:
# Load environment variables from .env file
load_dotenv()

True

In [19]:
# Get secrets from environment variables
connection_string = os.getenv('BLOB_CONNECTION_STRING')
container_name = os.getenv('BLOB_CONTAINER_NAME')
service_name = os.getenv('SEARCH_SERVICE_NAME')
key = os.getenv('SEARCH_KEY')
service_endpoint = os.getenv('SEARCH_ENDPOINT')
index = os.getenv('SEARCH_INDEX_NAME')

In [20]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [21]:
blob_list = container_client.list_blobs()
blob_map = {}

for blob in blob_list:
    blob_name = blob.name
    sas_token = generate_blob_sas(blob_service_client.account_name,
                                  container_name,
                                  blob_name,
                                  account_key=blob_service_client.credential.account_key,
                                  permission=BlobSasPermissions(read=True),
                                  expiry=datetime.now(timezone.utc) + timedelta(days=10)
                                )
    sas_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob_name}?{sas_token}"
    blob_map[os.path.splitext(blob_name)[0]] = sas_url
    
print(blob_map)    
    

{'Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_1': 'https://saaimcodemo.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_1.jpeg?se=2024-05-23T19%3A36%3A09Z&sp=r&sv=2024-05-04&sr=b&sig=ltJLkqukEnwb%2BX6Wd02qOaWAz3oGeFXSjoi5IZeW6KE%3D', 'Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_10': 'https://saaimcodemo.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_10.jpeg?se=2024-05-23T19%3A36%3A09Z&sp=r&sv=2024-05-04&sr=b&sig=EZN4G8KHmy0LNtPNlktJip81OHRisvlfJwWqdL1ILLA%3D', 'Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_11': 'https://saaimcodemo.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_11.jpeg?se=2024-05-23T19%3A36%3A09Z&sp=r&sv=2024-05-04&sr=b&sig=06w57lJjhLdZqPVNTNfauxSAlepTfP/z%2BU1TaoU8V

In [22]:
def update_blob_storage_url(chunk_id, blob_url):
 
    base_url=f"https://{service_name}.search.windows.net/indexes/{index}/docs/index?api-version=2020-06-30"
    headers = {"Content-Type": "application/json", "api-key": key}
    body = {
    "value": [
        {
            "@search.action": "merge",
            "chunk_id": chunk_id,
            "blob_url": blob_url
        }
    ]
    }
    requests.post(url = base_url, headers = headers, json = body)
 
 

In [28]:
search_client = SearchClient(service_endpoint, index, AzureKeyCredential(key))

In [30]:
results = search_client.search(search_text="*")
for result in results:
    chunk_id = result['chunk_id']
    title = os.path.splitext(result['title'])[0]
    blob_url = blob_map.get(title)
    update_blob_storage_url(chunk_id, blob_url)
    print(f'updating {title}: {blob_map.get(title)}')

updating Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_11: https://saaimcodemo.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_11.jpeg?se=2024-05-23T19%3A36%3A09Z&sp=r&sv=2024-05-04&sr=b&sig=06w57lJjhLdZqPVNTNfauxSAlepTfP/z%2BU1TaoU8VT0%3D
updating Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_1: https://saaimcodemo.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_1.jpeg?se=2024-05-23T19%3A36%3A09Z&sp=r&sv=2024-05-04&sr=b&sig=ltJLkqukEnwb%2BX6Wd02qOaWAz3oGeFXSjoi5IZeW6KE%3D
updating Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_2: https://saaimcodemo.blob.core.windows.net/fixedincomepdf-pages/Benchmark Barometer - Monthly FTSE Canada Bond Index Review - Apr 2023_page_2.jpeg?se=2024-05-23T19%3A36%3A09Z&sp=r&sv=2024-05-04&sr=b&sig=lAufdswFceemXZMK9mWRNoHQs0zcTs